# Convert .xml to pianoroll representation (pr)

conv .xml -> np (pianoroll) for 4 voices + 1 mix of all voices -> save with pickle 

folder structure: pianorolls -> voice_0 ,voice_1, voice_2, voice_3, voice_all -> voice_0_001.pkl 

### original conversion from humdrum to xml (in terminal)

In [232]:
#for FILE in *; do hum2xml -r $FILE > "${FILE%.*}.xml"; done

## Convert xml to pr for 1 file

In [1]:
import partitura
import numpy as np
import os
import pickle

In [20]:
name = "chor133.xml"


import_path = os.path.join("/Users/sayang/Masterproject/chorales_converted", name)
part = partitura.load_musicxml(import_path)
#print(part)

In [12]:
### Seperate the 4 different voices ####

part_zero = part[0]
part_one = part[1]
part_two = part[2]
part_three = part[3]

In [13]:
### Visualize the different voices ###

#print(part_one.pretty())   
#partitura.render(part_one) #still doesnt work

In [14]:
### maybe use time_uni = beat and time_div =12
### Convert every voice into a pianoroll numpy array ###

pr_zero = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
pr_zero = pr_zero.toarray()

pr_one = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
pr_one = pr_one.toarray()

pr_two = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
pr_two = pr_two.toarray()

pr_three = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
pr_three = pr_three.toarray()

pr = partitura.utils.compute_pianoroll(part, time_unit = "beat",time_div = 12)
pr = pr.toarray()

In [15]:
### print the shape of the np arrays ###
print(pr_zero.shape)
print(pr_one.shape)
print(pr_two.shape)
print(pr_three.shape)
print(pr.shape)

(128, 756)
(128, 756)
(128, 756)
(128, 756)
(128, 756)


In [16]:
### See if the sum of the individual represenations are exaclty the same as the original ###
### - if not set all values bigger than 1 to 0 again ###

print(np.sum(pr_zero))
print(np.sum(pr_one))
print(np.sum(pr_two))
print(np.sum(pr_three))

print("sum of all 4 voice:", np.sum(pr_zero+pr_one+pr_two+pr_three))
print("sum of mixed pr representation:",np.sum(pr))

756
756
756
756
sum of all 4 voice: 3024
sum of mixed pr representation: 3012


## set all values > 1 in the composition of the 4 seperate voices to 1 ###



In [17]:
pr_comp = pr_zero+pr_one+pr_two+pr_three
pr_comp =  np.where(pr_comp < 1 , pr_comp, 1) 

quick check:

In [18]:
print( np.sum(pr_comp) == np.sum(pr))

True


## save the data   

save every voice in a seperate folder as well as mixed representation the naming scheme is 
 e.g. pianoroll/voice_0/voice_0_371.pkl

In [199]:
#example
os.path.join("pianoroll", voice_folder[0], name)

'pianoroll/voice_0/chor001.xml'

In [205]:
file_out_names = [ "voice_0_" + name[4:7]+ ".pkl" , "voice_1_" + name[4:7] + ".pkl" , "voice_2_" + name[4:7] , "voice_3_" + name[4:7] + ".pkl", "voice_all_" + name[4:7]+".pkl"   ]

In [197]:
voice_folder = ["voice_0","voice_1","voice_2","voice_3","voice_all"]
pianorolls = [pr_zero,pr_one,pr_two,pr_three,pr]


for n,i in enumerate(pianorolls):
    a_folder = voice_folder[n]
    a_file = file_out_names[n]
    joined_path = os.path.join("pianoroll", a_folder, a_file)
    with open(joined_path, 'wb') as f:
        pickle.dump(i, f)

## test - loading pickle file again

In [198]:
with open("pianoroll/voice_0/voice_0_371.pkl",'rb') as f:
    loaded_obj = pickle.load(f)
    print(loaded_obj.shape)

(128, 576)


# Now do this for the whole folder

In [23]:
path = "/Users/sayang/Masterproject/chorales_converted"

for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    part = partitura.load_musicxml(fullname)
        
    part_zero = part[0]
    part_one = part[1]
    part_two = part[2]
    part_three = part[3]
        
    
    pr_zero = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
    pr_zero = pr_zero.toarray()

    pr_one = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
    pr_one = pr_one.toarray()

    pr_two = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
    pr_two = pr_two.toarray()

    pr_three = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
    pr_three = pr_three.toarray()

    pr = partitura.utils.compute_pianoroll(part, time_unit = "beat",time_div = 12)
    pr = pr.toarray()
    
    if pr_zero.shape == pr_one.shape == pr_two.shape == pr_three.shape == pr.shape : 
        pr_comp = pr_zero+pr_one+pr_two+pr_three
        pr_comp =  np.where(pr_comp < 1 , pr_comp, 1) 

        file_out_names = [ "voice_0_" + filename[4:7]+ ".pkl" , "voice_1_" + filename[4:7] + ".pkl" , "voice_2_" + filename[4:7] + ".pkl", "voice_3_" + filename[4:7] + ".pkl", "voice_all_" + filename[4:7]+".pkl"   ]    
        voice_folder = ["voice_0","voice_1","voice_2","voice_3","voice_all"]
        pianorolls = [pr_zero,pr_one,pr_two,pr_three,pr]


        for n,i in enumerate(pianorolls):
            a_folder = voice_folder[n]
            a_file = file_out_names[n]
            joined_path = os.path.join("pianoroll", a_folder, a_file)
            #if a_file not in  ["chor133.xml", "chor043.xml", "chor283.xml"]:
            with open(joined_path, 'wb') as f:
                pickle.dump(i, f)    
                
    else:
        print(filename)

chor133.xml
chor043.xml
chor283.xml


# Test pr col size by loading 2 measure piece in 4/4 with random notes

In [33]:
name = "test2.musicxml"

import_path = os.path.join("/Users/sayang/Masterproject", name)
part_group = partitura.load_musicxml(import_path)
#print(part)
part = part_group.children[0]

In [34]:
test123 = partitura.utils.compute_pianoroll(part, time_unit = "beat",time_div = 12)
test123 = test123.toarray()
test123.shape

(128, 72)

In [31]:
part.note_array

array([(0., 1., 0., 1., 0, 1, 70, 1, 'None'),
       (4., 1., 4., 1., 4, 1, 74, 1, 'None')],
      dtype=[('onset_beat', '<f4'), ('duration_beat', '<f4'), ('onset_quarter', '<f4'), ('duration_quarter', '<f4'), ('onset_div', '<i4'), ('duration_div', '<i4'), ('pitch', '<i4'), ('voice', '<i4'), ('id', '<U256')])